# Horologium

### Creating a digital clock for Roman-style date-and-timekeeping, anywhere in the world. 
-------------------------------
Jacob Schwartz

Professor Hitchner

Intro to Digital Humanities

May 2, 2025


For this assignment, my objective was to create a digital clock for the Romans, who had a way of reckoning time based simply on the movement of the sun. 

## Roman Timekeeping
Let's start with how the Romans told time over the course of a day. One of our best sources for this is Censorinus in *De Die Natali (Liber),* written around 238 CE.<sup>1</sup> According to him, *in horas XII diem divisum esse noctemque in totidem vulgo notum est* ("that the day and night are [each] divided into twelve hours is known among all the people").<sup>2</sup> The "day" (*naturaliter dies*) is reckoned from sunrise to sunset, while the "night" (*nox*) is sunset to sunrise.<sup>3</sup> As the day lengthens and shortens with the changing of the seasons, so too do the hours: we have twelve short hours daylight in the winter and twelve long hours in the summer. In addition, the length of these hours varies based on latitude - a soldier on Hadrian's wall in wintertime, for instance, would experience far shorter hours than his compatriot in Alexandria on the same day.<sup>4</sup>

---------

1: https://elfinspell.com/ClassicalTexts/Maude/Censorinus/DeDieNatale-Part2.html#topref107

2: https://penelope.uchicago.edu/Thayer/L/Roman/Texts/Censorinus/text*.html

3: https://penelope.uchicago.edu/Thayer/L/Roman/Texts/Censorinus/text*.html

4: https://penelope.uchicago.edu/Thayer/E/Roman/Texts/secondary/SMIGRA*/Hora.html

## Calculating the Day
To calculate 12 equal hours from sunrise to sunset, then, we will need two thing. The first SunriseSunset.io, an API which finds the time of sunrise and sunset for a given latitude and longitude. The second thing we will need is the datetime module, which will allow us to get the current date and time as well as calculate the difference between times (called a timedelta). 

In [11]:
# First, we'll import our modules.
import datetime
from datetime import timezone
import requests

# Next, we're going to define our time now. Later, we'll use the module to get the current time, but for now, let's say it's 
# 12:00 PM on March 15th (the Ides!). 
timenow = datetime.datetime(2025, 3, 15, 12, 0, 0, 000000)


# Then we'll isolate the date and convert it into a string. We're going to need to use this a lot. 
thedate = timenow.date().strftime('%Y-%m-%d')

# These look like this:
print(timenow)
print(thedate)

2025-03-15 12:00:00
2025-03-15


In [12]:
# Now let's use our API to find the sunrise and sunset at a given location. For now, let's pick a spot in London
# for ease of use with UTC. Let's use the City of London, the neighborhood built on the remains of the ancient city of Londinium.
lat = '51.515556'
lng = '-0.093056'

# Powered by SunriseSunset.io: https://sunrisesunset.io/api/
def make_sun_request(lat, lng):
    url = f"https://api.sunrisesunset.io/json?lat={lat}&lng={lng}&timezone=UTC&date={thedate}&time_format=24"
    r = requests.get(url)
    response = r.json()
    sunrise = response['results']['sunrise']
    sunset = response['results']['sunset']

    return [sunrise, sunset]

make_sun_request(lat, lng)

['06:15:50', '18:05:28']

In [13]:
# Since these are strings, we need to convert them to datetimes to do calculations with them. 
def convert_sun_to_datetime(sunrise, sunset):
    risetime = datetime.datetime.strptime(f"{thedate}, {sunrise}", "%Y-%m-%d, %H:%M:%S")
    settime = datetime.datetime.strptime(f"{thedate}, {sunset}", "%Y-%m-%d, %H:%M:%S")
    return [risetime, settime]

def get_datetimes_from_coord(lat, lng):
    srss = make_sun_request(lat, lng)
    sunrise = srss[0]
    sunset = srss[1]
    datetimes = convert_sun_to_datetime(sunrise, sunset)
    return datetimes

datetimes = get_datetimes_from_coord(lat, lng)
datetimes

[datetime.datetime(2025, 3, 15, 6, 15, 50),
 datetime.datetime(2025, 3, 15, 18, 5, 28)]

In [15]:
# Now we can calculate our twelve hours of the day. 
def calc_hodie(risetime, settime):
    dies = settime-risetime
    hora = dies/12
    hora_prima = risetime
    hora_secunda = hora_prima+hora
    hora_tertia = hora_prima+hora*2
    hora_quarta = hora_prima+hora*3
    hora_quinta = hora_prima+hora*4
    hora_sexta = hora_prima+hora*5
    hora_septima = hora_prima+hora*6
    hora_octava = hora_prima+hora*7
    hora_nona = hora_prima+hora*8
    hora_decima = hora_prima+hora*9
    hora_undecima = hora_prima+hora*10
    hora_duodecima = hora_prima+hora*11
    if hora_prima <= timenow < hora_secunda:
        tempus = "\nI\nprima diei hora"
    elif hora_secunda <= timenow < hora_tertia:
        tempus = "\nII\nseconda diei hora"
    elif hora_tertia <= timenow < hora_quarta:
        tempus = "\nIII\ntertia diei hora"
    elif hora_quarta <= timenow < hora_quinta:
        tempus = "\nIV\nquarta diei hora"
    elif hora_quinta <= timenow < hora_sexta:
        tempus = "\nV\nquinta diei hora"
    elif hora_sexta <= timenow < hora_septima:
        tempus = "\nVI\nsexta diei hora"
    elif hora_septima <= timenow < hora_octava:
        tempus = "\nVII\nseptima diei hora"
    elif hora_octava <= timenow < hora_nona:
        tempus = "\nVIII\noctava diei hora"
    elif hora_nona <= timenow < hora_decima:
        tempus = "\nIX\nnona diei hora"
    elif hora_decima <= timenow < hora_undecima:
        tempus = "\nX\ndecima diei hora"
    elif hora_undecima <= timenow < hora_duodecima:
        tempus = "\nXI\nundecima diei hora"
    elif hora_duodecima <= timenow < settime:
        tempus = "\nXII\nduodecima diei hora"
    else:
        tempus = "Eheu! Couldn't find the hour of the day!"
    return tempus

risetime = datetimes[0]
settime = datetimes[1]
tempus = calc_hodie(risetime, settime)

print(tempus)


VI
sexta diei hora


## Calculating the Night
Before we get to calculating the time between sunset and sunrise, we need to talk about some other ways of reckoning time among the Romans. In particular, Censorinus mentions that *alii diem quadripertito, sed et noctem similiter dividebant; idque similitudo testatur militaris, ubi dicitur vigilia prima, item secunda et tertia et quarta* ("others, with the day having been divided in four, even divided the night similarly; and this is witnessed by the similar [divisions] in military language, where "first watch," is said, then "second" and "third" and "fourth").<sup>5</sup> The four watches, particularly those of the night, are an especially significant alternative, appearing frequently in the New Testament as well.<sup>6</sup> For this reason, we're going to include them as well, with each watch being three hours long.<sup>7</sup>

In order to achieve this

-----------

5:https://penelope.uchicago.edu/Thayer/L/Roman/Texts/Censorinus/text*.html

6: Matthew 14:25, Luke 12:38, Mark 13:35

7:https://books.google.com/books?id=-tdRAAAAcAAJ&printsec=frontcover&source=gbs_ge_summary_r&cad=0#v=onepage&q&f=false page 307.

## Bibliography
https://elfinspell.com/ClassicalTexts/Maude/Censorinus/DeDieNatale-Part2.html#topref107
https://penelope.uchicago.edu/Thayer/L/Roman/Texts/Censorinus/text*.html
https://penelope.uchicago.edu/Thayer/E/Roman/Texts/secondary/SMIGRA*/Hora.html